<a href="https://colab.research.google.com/github/heejinohn/lzho/blob/main/Reddit_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment

In [31]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 163 kB in 2s (67.0 kB/s)
Reading

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


In [39]:
set(['J','F','M','A','M','J','J','A','S','O','N','D'])

{'A', 'D', 'F', 'J', 'M', 'N', 'O', 'S'}

In [32]:
url = 'https://www.reddit.com/r/wallstreetbets/search/?q=flair%3A%22Daily%20Discussion%22&restrict_sr=1&sort=new'
driver.get(url)

In [41]:
from datetime import date, timedelta
from dateutil.parser import parse
import re

In [34]:
yesterday = date.today() - timedelta(days=1)
links = driver.find_elements_by_xpath('//*[@class="_eYtD2XCVieq6emjKBH3m"]')

r/wallstreetbet was created on January 31, 2012

In [71]:
pattern = re.compile(r'[ADFJMNOS][a-z]+ \d{1,2},? \d{4}') ## pattern to find date

for a in links:
    if a.text.startswith('Daily Discussion Thread'):
        z = re.search("[ADFJMNOS][a-z]+ \d{1,2},? \d{4}", a.text)
        if z:
            date = z.group()
    parsed = parse(date) 
    if parse(str(yesterday)) == parsed:
        link = a.find_element_by_xpath('../..').get_attribute('href')
        
        ## Not sure if this part is valid. There's a new "Weekend" flair
    if a.text.startswith('Weekend'):
        weekend_date = a.text.split(' ')
        parsed_date = weekend_date[-3] + ' ' + \
            weekend_date[-2].split("-")[1] + weekend_date[-1]
        parsed = parse(parsed_date) 
        saturday = weekend_date[-3] + ' ' + \
            str(int(weekend_date[-2].split("-")[1]
            .replace(',','')) - 1) + ' ' + weekend_date[-1] 

    if parse(str(yesterday)) == parsed: 
        link = a.find_element_by_xpath('../..').get_attribute('href')
    elif parse(str(yesterday)) == parse(str(saturday)):
        link = a.find_element_by_xpath('../..').get_attribute('href')

NameError: ignored

In [ ]:
### r/wallstreetbet has a flair summary. Check to determine what will be most 
### suitable.